In [1]:
import os
from datasets_gta5 import GTA5
import albumentations as A
import torch

GTA5_PATH = '/home/arda/.cache/kagglehub/datasets/ardaerendoru/gtagta/versions/1/GTA5/GTA5'
GTA5_IMAGES = os.path.join(GTA5_PATH, 'images')
GTA5_LABELS = os.path.join(GTA5_PATH, 'labels')

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

In [12]:

transform = A.Compose([
    A.Resize(512, 1024)
    # A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
GTA5_dataset = GTA5(GTA5_path=GTA5_PATH, transform=transform)



state_dict = resnet50.state_dict()

print(ckp.keys())


odict_keys(['conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'layer1.0.conv1.weight', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.bn1.running_mean', 'layer1.0.bn1.running_var', 'layer1.0.bn1.num_batches_tracked', 'layer1.0.conv2.weight', 'layer1.0.bn2.weight', 'layer1.0.bn2.bias', 'layer1.0.bn2.running_mean', 'layer1.0.bn2.running_var', 'layer1.0.bn2.num_batches_tracked', 'layer1.0.conv3.weight', 'layer1.0.bn3.weight', 'layer1.0.bn3.bias', 'layer1.0.bn3.running_mean', 'layer1.0.bn3.running_var', 'layer1.0.bn3.num_batches_tracked', 'layer1.0.downsample.0.weight', 'layer1.0.downsample.1.weight', 'layer1.0.downsample.1.bias', 'layer1.0.downsample.1.running_mean', 'layer1.0.downsample.1.running_var', 'layer1.0.downsample.1.num_batches_tracked', 'layer1.1.conv1.weight', 'layer1.1.bn1.weight', 'layer1.1.bn1.bias', 'layer1.1.bn1.running_mean', 'layer1.1.bn1.running_var', 'layer1.1.bn1.num_batches_tracked', 'layer1.1.conv2.we

In [13]:
print(state_dict.keys())

odict_keys(['stem.conv1.weight', 'stem.conv1.norm.weight', 'stem.conv1.norm.bias', 'stem.conv1.norm.running_mean', 'stem.conv1.norm.running_var', 'stem.conv1.norm.num_batches_tracked', 'res2.0.shortcut.weight', 'res2.0.shortcut.norm.weight', 'res2.0.shortcut.norm.bias', 'res2.0.shortcut.norm.running_mean', 'res2.0.shortcut.norm.running_var', 'res2.0.shortcut.norm.num_batches_tracked', 'res2.0.conv1.weight', 'res2.0.conv1.norm.weight', 'res2.0.conv1.norm.bias', 'res2.0.conv1.norm.running_mean', 'res2.0.conv1.norm.running_var', 'res2.0.conv1.norm.num_batches_tracked', 'res2.0.conv2.weight', 'res2.0.conv2.norm.weight', 'res2.0.conv2.norm.bias', 'res2.0.conv2.norm.running_mean', 'res2.0.conv2.norm.running_var', 'res2.0.conv2.norm.num_batches_tracked', 'res2.0.conv3.weight', 'res2.0.conv3.norm.weight', 'res2.0.conv3.norm.bias', 'res2.0.conv3.norm.running_mean', 'res2.0.conv3.norm.running_var', 'res2.0.conv3.norm.num_batches_tracked', 'res2.1.conv1.weight', 'res2.1.conv1.norm.weight', 'res2.

In [3]:
from models import CustomResNet
encoder = CustomResNet(pretrained=False)
encoder.load_state_dict(torch.load('/home/arda/dinov2/distillation/checkpoints/best_model.pth')['student_state_dict'])

encoder.eval()
encoder.to(device)


/storage/disk0/arda/dinov2/distillation/../../dinov2/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/storage/disk0/arda/dinov2/distillation/../../dinov2/dinov2/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/storage/disk0/arda/dinov2/distillation/../../dinov2/dinov2/layers/block.py:33: UserWarning: xFormers is available (Block)
  warnings.warn("xFormers is available (Block)")


CustomResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), strid

In [4]:
asd = torch.randn(1, 3, 512, 1024).to(device)
encoder(asd)["feature_map"].shape

# Freeze all parameters of the encoder
for param in encoder.parameters():
    param.requires_grad = False

In [5]:
# First, let's create a simple decoder network
import numpy as np
import tqdm as tqdm
class SegmentationDecoder(torch.nn.Module):
    def __init__(self, in_channels=2048, num_classes=19):
        super().__init__()
        self.decoder = torch.nn.Sequential(
            # 16x32 -> 32x64
            torch.nn.ConvTranspose2d(in_channels, 1024, kernel_size=4, stride=2, padding=1),
            torch.nn.BatchNorm2d(1024),
            torch.nn.ReLU(),
            
            # 32x64 -> 64x128
            torch.nn.ConvTranspose2d(1024, 512, kernel_size=4, stride=2, padding=1),
            torch.nn.BatchNorm2d(512),
            torch.nn.ReLU(),
            
            # 64x128 -> 128x256
            torch.nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            torch.nn.BatchNorm2d(256),
            torch.nn.ReLU(),
            
            # 128x256 -> 256x512
            torch.nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            torch.nn.BatchNorm2d(128),
            torch.nn.ReLU(),
            
            # 256x512 -> 512x1024
            torch.nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            torch.nn.BatchNorm2d(64),
            torch.nn.ReLU(),
            
            # Final 1x1 conv to get to num_classes
            torch.nn.Conv2d(64, num_classes, kernel_size=1)
        )
    def forward(self, x):
        x = self.decoder(x)
        # Ensure exact output size
        if x.shape[-2:] != (512, 1024):
            x = torch.nn.functional.interpolate(
                x, size=(512, 1024), 
                mode='bilinear', 
                align_corners=False
            )
        return x

# Initialize decoder, optimizer, and loss function
decoder = SegmentationDecoder().to(device)
optimizer = torch.optim.Adam(decoder.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss(ignore_index=255)

def fast_hist(a: np.ndarray, b: np.ndarray, n: int) -> np.ndarray:
    k = (b >= 0) & (b < n)
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n ** 2).reshape(n, n)

def per_class_iou(hist: np.ndarray) -> np.ndarray:
    epsilon = 1e-5
    return (np.diag(hist)) / (hist.sum(1) + hist.sum(0) - np.diag(hist) + epsilon)

def train_epoch(encoder, decoder, dataloader, optimizer, criterion, device, num_classes=19):
    decoder.train()
    encoder.eval()  # Keep DINO frozen
    
    total_loss = 0
    hist = np.zeros((num_classes, num_classes))  # Single histogram for entire epoch
    total_pixels = 0
    correct_pixels = 0
    
    for images, labels in tqdm.tqdm(dataloader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Get DINO features
        with torch.no_grad():
            features = encoder(images)["feature_map"]
        
        # Forward pass through decoder
        outputs = decoder(features)
        
        # Resize outputs to match label size if needed
        if outputs.shape[-2:] != labels.shape[-2:]:
            outputs = torch.nn.functional.interpolate(
                outputs, size=labels.shape[-2:], mode='bilinear', align_corners=False)
        
        # Calculate loss
        loss = criterion(outputs, labels)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        # Calculate metrics
        preds = torch.argmax(torch.softmax(outputs, dim=1), dim=1)
        
        # Pixel Accuracy
        valid_mask = labels != 255  # Ignore index
        total_pixels += valid_mask.sum().item()
        correct_pixels += ((preds == labels) & valid_mask).sum().item()
        
        # IoU
        preds = preds.cpu().numpy()
        target = labels.cpu().numpy()
        hist += fast_hist(preds.flatten(), target.flatten(), num_classes)
    
    # Calculate final metrics
    pixel_acc = correct_pixels / total_pixels
    
    # Per-class accuracy (mean class accuracy)
    class_acc = np.diag(hist) / (hist.sum(1) + np.finfo(np.float32).eps)
    mean_class_acc = np.nanmean(class_acc)
    
    # IoU metrics
    iou = per_class_iou(hist)
    mean_iou = np.nanmean(iou)
    
    metrics = {
        'loss': total_loss / len(dataloader),
        'pixel_acc': pixel_acc,
        'mean_class_acc': mean_class_acc,
        'mean_iou': mean_iou,
        'class_iou': iou,
        'class_acc': class_acc
    }
    
    return metrics

train_loader = torch.utils.data.DataLoader(
    GTA5_dataset, 
    batch_size=4,
    shuffle=True,
    num_workers=4
)
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    metrics = train_epoch(encoder, decoder, train_loader, optimizer, criterion, device)
    
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print(f"Loss: {metrics['loss']:.4f}")
    print(f"Pixel Accuracy: {metrics['pixel_acc']:.4f}")
    print(f"Mean Class Accuracy: {metrics['mean_class_acc']:.4f}")
    print(f"Mean IoU: {metrics['mean_iou']:.4f}")
    
    # Optionally print per-class metrics
    print("\nPer-class metrics:")
    for i in range(19):  # Assuming 19 classes
        print(f"Class {i:2d} - Acc: {metrics['class_acc'][i]:.4f}, IoU: {metrics['class_iou'][i]:.4f}")

100%|██████████| 625/625 [04:56<00:00,  2.11it/s]



Epoch 1/10
Loss: 0.9172
Pixel Accuracy: 0.8430
Mean Class Accuracy: 0.3394
Mean IoU: 0.2454

Per-class metrics:
Class  0 - Acc: 0.9612, IoU: 0.9330
Class  1 - Acc: 0.5427, IoU: 0.3984
Class  2 - Acc: 0.6960, IoU: 0.6486
Class  3 - Acc: 0.4081, IoU: 0.1877
Class  4 - Acc: 0.0098, IoU: 0.0003
Class  5 - Acc: 0.0248, IoU: 0.0004
Class  6 - Acc: 0.0015, IoU: 0.0003
Class  7 - Acc: 0.0008, IoU: 0.0001
Class  8 - Acc: 0.6637, IoU: 0.5252
Class  9 - Acc: 0.7468, IoU: 0.3243
Class 10 - Acc: 0.9077, IoU: 0.8505
Class 11 - Acc: 0.0021, IoU: 0.0001
Class 12 - Acc: 0.0004, IoU: 0.0002
Class 13 - Acc: 0.7488, IoU: 0.5134
Class 14 - Acc: 0.5864, IoU: 0.2540
Class 15 - Acc: 0.1099, IoU: 0.0164
Class 16 - Acc: 0.0373, IoU: 0.0093
Class 17 - Acc: 0.0004, IoU: 0.0003
Class 18 - Acc: 0.0001, IoU: 0.0001


100%|██████████| 625/625 [04:55<00:00,  2.11it/s]



Epoch 2/10
Loss: 0.4546
Pixel Accuracy: 0.8839
Mean Class Accuracy: 0.4948
Mean IoU: 0.3101

Per-class metrics:
Class  0 - Acc: 0.9690, IoU: 0.9489
Class  1 - Acc: 0.6911, IoU: 0.5275
Class  2 - Acc: 0.7780, IoU: 0.7190
Class  3 - Acc: 0.6241, IoU: 0.3933
Class  4 - Acc: 0.6419, IoU: 0.0075
Class  5 - Acc: 0.4608, IoU: 0.0332
Class  6 - Acc: 0.0000, IoU: 0.0000
Class  7 - Acc: 0.0000, IoU: 0.0000
Class  8 - Acc: 0.7680, IoU: 0.6387
Class  9 - Acc: 0.7430, IoU: 0.5379
Class 10 - Acc: 0.9224, IoU: 0.8818
Class 11 - Acc: 0.0000, IoU: 0.0000
Class 12 - Acc: 0.1276, IoU: 0.0008
Class 13 - Acc: 0.7993, IoU: 0.6953
Class 14 - Acc: 0.6120, IoU: 0.4460
Class 15 - Acc: 0.7237, IoU: 0.0413
Class 16 - Acc: 0.4218, IoU: 0.0206
Class 17 - Acc: 0.1176, IoU: 0.0000
Class 18 - Acc: 0.0000, IoU: 0.0000


100%|██████████| 625/625 [04:56<00:00,  2.11it/s]


...

100%|██████████| 625/625 [04:54<00:00,  2.12it/s]



Epoch 9/10
Loss: 0.2384
Pixel Accuracy: 0.9212
Mean Class Accuracy: 0.7461
Mean IoU: 0.5463

Per-class metrics:
Class  0 - Acc: 0.9792, IoU: 0.9651
Class  1 - Acc: 0.8003, IoU: 0.6570
Class  2 - Acc: 0.8676, IoU: 0.8046
Class  3 - Acc: 0.7646, IoU: 0.5680
Class  4 - Acc: 0.6511, IoU: 0.3756
Class  5 - Acc: 0.6094, IoU: 0.2464
Class  6 - Acc: 0.6011, IoU: 0.1955
Class  7 - Acc: 0.7666, IoU: 0.4080
Class  8 - Acc: 0.8227, IoU: 0.7088
Class  9 - Acc: 0.8097, IoU: 0.6568
Class 10 - Acc: 0.9464, IoU: 0.9122
Class 11 - Acc: 0.6510, IoU: 0.3866
Class 12 - Acc: 0.7656, IoU: 0.0348
Class 13 - Acc: 0.8730, IoU: 0.7972
Class 14 - Acc: 0.8622, IoU: 0.7193
Class 15 - Acc: 0.9052, IoU: 0.7095
Class 16 - Acc: 0.8661, IoU: 0.7244
Class 17 - Acc: 0.6333, IoU: 0.5092
Class 18 - Acc: 0.0000, IoU: 0.0000


100%|██████████| 625/625 [04:52<00:00,  2.13it/s]


Epoch 10/10
Loss: 0.2280
Pixel Accuracy: 0.9239
Mean Class Accuracy: 0.7501
Mean IoU: 0.5702

Per-class metrics:
Class  0 - Acc: 0.9801, IoU: 0.9665
Class  1 - Acc: 0.8071, IoU: 0.6662
Class  2 - Acc: 0.8732, IoU: 0.8117
Class  3 - Acc: 0.7767, IoU: 0.5867
Class  4 - Acc: 0.6618, IoU: 0.3966
Class  5 - Acc: 0.6204, IoU: 0.2616
Class  6 - Acc: 0.6247, IoU: 0.2174
Class  7 - Acc: 0.7690, IoU: 0.4349
Class  8 - Acc: 0.8273, IoU: 0.7146
Class  9 - Acc: 0.8156, IoU: 0.6670
Class 10 - Acc: 0.9482, IoU: 0.9142
Class 11 - Acc: 0.6630, IoU: 0.3984
Class 12 - Acc: 0.7089, IoU: 0.2898
Class 13 - Acc: 0.8784, IoU: 0.8042
Class 14 - Acc: 0.8755, IoU: 0.7419
Class 15 - Acc: 0.8709, IoU: 0.6983
Class 16 - Acc: 0.8630, IoU: 0.7175
Class 17 - Acc: 0.6885, IoU: 0.5459
Class 18 - Acc: 0.0000, IoU: 0.0000


: 

In [6]:
# Epoch 3/10
# Loss: 1.2027
# Pixel Accuracy: 0.6440
# Mean Class Accuracy: 0.1213
# Mean IoU: 0.0818

# Per-class metrics:
# Class  0 - Acc: 0.6736, IoU: 0.6670
# Class  1 - Acc: 0.1000, IoU: 0.0000
# Class  2 - Acc: 0.3823, IoU: 0.1999
# Class  3 - Acc: 0.0000, IoU: 0.0000
# Class  4 - Acc: 0.0000, IoU: 0.0000
# Class  5 - Acc: 0.0000, IoU: 0.0000
# Class  6 - Acc: 0.0000, IoU: 0.0000
# Class  7 - Acc: 0.0000, IoU: 0.0000
# Class  8 - Acc: 0.4519, IoU: 0.0889
# Class  9 - Acc: 0.0000, IoU: 0.0000
# Class 10 - Acc: 0.6970, IoU: 0.5976
# Class 11 - Acc: 0.0000, IoU: 0.0000
# Class 12 - Acc: 0.0000, IoU: 0.0000
# Class 13 - Acc: 0.0000, IoU: 0.0000
# Class 14 - Acc: 0.0000, IoU: 0.0000
# Class 15 - Acc: 0.0000, IoU: 0.0000
# Class 16 - Acc: 0.0000, IoU: 0.0000
# Class 17 - Acc: 0.0000, IoU: 0.0000
# Class 18 - Acc: 0.0000, IoU: 0.0000

In [7]:
import matplotlib.pyplot as plt
from datasets import get_id_to_color   


img_idx = 310

embeddings = encoder(GTA5_dataset[img_idx][0].unsqueeze(0).to(device), n=1, reshape=True, return_class_token=False, norm=False)[0]
out = decoder(embeddings)
id_to_color = get_id_to_color()

pred = out.argmax(1).cpu().numpy()
pred = pred.reshape(518, 1036)
# Convert class IDs to RGB colors
color_map = np.array([id_to_color.get(i, (0, 0, 0)) for i in range(max(id_to_color.keys()) + 1)])
pred_rgb = color_map[pred]

plt.figure(figsize=(10, 10))
plt.imshow(pred_rgb)

plt.figure(figsize=(10, 10))
labels = GTA5_dataset[img_idx][1].cpu().numpy()
color_map = np.array([id_to_color.get(i, (0, 0, 0)) for i in range(max(id_to_color.keys()) + 1)])
pred_rgb = np.zeros((*labels.shape, 3), dtype=np.uint8)
mask = labels < len(color_map)
pred_rgb[mask] = color_map[labels[mask]]
plt.imshow(pred_rgb)


ImportError: cannot import name 'get_id_to_color' from 'datasets' (/home/arda/miniconda3/envs/dinov2/lib/python3.9/site-packages/datasets/__init__.py)

In [28]:
GTA5_dataset[img_idx][1].shape


torch.Size([518, 1036])